Projet d'optimisation : effacement de consommation
=

**1. Étude du problème d'optimisation**

1. La facture s'écrit comme $\int_{t_0}^{t_f} c(t)P(t) \,dt = \sum_{i=0}^{N} \int_{t_i}^{t_i + \Delta t} c(t)P(t) \,dt$.\
Or, sur un intervalle $\Delta t$, le coût par unité de puissance et de temps est constant puisque la puissance ne varie pas.\
La facture s'écrit donc : $\sum_{i=0}^{N} c_iP_i\Delta t = \Delta t \sum_{i=0}^{N} c_iP_i$

2. L'équation (2) comprend les échanges thermiques avec l'extérieur ainsi que la chaleur apportée par le chauffage de l'immeuble. Le premier terme permet en outre de prendre en compte l'inertie thermique du batiment. La température s'écrit donc comme la solution d'une équation différentielle dont la condition initiale est la température à l'intervalle $\Delta t$ précédent.

3. Le problème d'optimisation, s'écrit, avec une fonction f à minimiser qui correspond au coût de la facture, à un facteur dt près qu'on omettra (le probleème de minimisation reste alors inchangé).
Ainsi, la variable d'action x sera le vecteur des puissances $P_0,P_1, \ldots, P_N$ et des températures $T_0,T_1, \ldots, T_N$ dans cet ordre: 
$$x= (P_0,P_1, \ldots, P_N,T_0,T_1, \ldots, T_N$$
Et la fonction à minimiser:
$$f(x)=\sum_{i=0}^N c_iP_i=\sum_{i=0}^N c[i]*x[i]=\mathbf{c}^\intercal \mathbf{x}$$ 
Avec $c=(c_0,c_1, \ldots, c_N, 0,\ldots, 0)$, donc $c[i]=\left\{\begin{array}{rcl} c_i & \mbox{si} & 0 \leq i \leq N \\ 0 & \mbox{si} & N+1 \leq i \leq 2N+1 \end{array}\right.$

Les contraintes égalité et inégalité sont définies comme suit:
$$c_{eq}(x)= \left\{\begin{array}{rcl} 0 & \mbox{si} & 0 \leq i \leq N-1 \\ x[N] & \mbox{si} & i=N \\x[N+1] - T_{in} & \mbox{si} & i=N+1\\ e^{-(k+h)dt} x[i-1] +\frac{(1-e^{-(k+h)dt}}{k+h} (b*x[i-N-2]+hTe) \mbox{si} & N+2 \leq i \leq 2N+1 \end{array}\right.$$

D'autre part: 
$$c_{in}(x)=(u,v)$$ 
Avec : 
$u= x - (P_M,\ldots, P_M,T_M,\ldots,T_M)^T = x - w^T $ et $ w=\begin{pmatrix} P_M  \\ \ldots\\ P_M \\T_M\\ \vdots\\T_M \end{pmatrix}=((P_M)_{0\leq i \leq N}, (T_M)_{N+1\leq i \leq 2N+1})$

et $v= z^T - x$ où $z=\begin{pmatrix} 0\\ \ldots\\0\\T_m\\ \vdots\\T_m \end{pmatrix}=((0)_{0\leq i \leq N}, (T_m)_{N+1\leq i \leq 2N+1})$


4. La fonction à minimiser est convexe (simple produit scalaire).\
La contrainte inégalité est également convexe (fonction linéaire en x).

In [27]:
import numpy as np
from scipy import optimize
from casadi import *
import time
import autograd

In [5]:
N=47 
dt=0.5
t0=23
k=0.01
h=0.05
b=1/500

In [6]:
Tm=18
T_in=Tm
TM=30
PM=5000
c_cr=1
c_pl=3/2

In [7]:
c=np.full(2*N+2, c_pl)

c[2:15]=c_cr
c[26:31]=c_cr
c[48:] = 0
print(c)

[1.5 1.5 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.5 1.5 1.5
 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.  1.  1.  1.  1.  1.5 1.5 1.5 1.5 1.5
 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0. ]


In [18]:
def f(x):
    return np.dot(c,x)
    
def c_eq(x):
    c=np.zeros(2*N+2)
    c[N]=x[N]
    c[N+1]= x[N+1] - T_in
    t=t0
    
    for i in range(N+2,2*N+2):
        Te = 4+ 8*np.exp(-(t-12)**2/40)
        g = np.exp(-(k+h)*dt)*x[i-1] +(1-np.exp(-(k+h)*dt))(b*x[i-1-(N+1)]+h*Te)/(k+h)
        c[i] = x[i]- g
        t += dt
    return c

def c_in(x):
    u=np.zeros(2*N+2)
    v=np.zeros(2*N+2)
    cin=np.zeros((2*N+2,2))
    for i in range(N+1, 2*N+2):
        if (16+N+1<=i and i<=20+N+1):
            v[i]=Tm - x[i]
            u[i]=x[i]- TM
        else:
            if (38+N+1<=i and i<=47+N+1):
                v[i]=Tm - x[i]
                u[i]=x[i]- TM
    for i in range(0,N+1):
        u[i]=x[i]- PM
        v[i]= - x[i]
    cin[:,0]=u
    cin[:,1]=v
    return cin

def u(x, i):
    if (16+N+1<=i and i<=20+N+1) or (38+N+1<=i and i<=47+N+1):
        return x[i]- TM
    elif i in range(0, N+1):
        return x[i] - PM
    else : 
        return 0

def v(x, i):
    if (16+N+1<=i and i<=20+N+1) or (38+N+1<=i and i<=47+N+1):
        return Tm - x[i]
    elif i in range(0, N+1):
        return -x[i]
    else :
        return 0

##### Avec Casadi

In [26]:
opti = casadi.Opti()
x = opti.variable(2*N+2)
f=0
for i in range(n):
    f+=x[i]*c[i]

def u(x):
    return x[0]

def v(x):
    return x[0]

def c_eq(x):
    return 0
    
opti.minimize(f)
for i in range(0, 2*N+2):
    opti.subject_to(u(x) <= 0)
    opti.subject_to(v(x) <= 0)
    
#opti.subject_to(c_eq(x) == 0)

x0 = np.zeros(2*N+2)
x[N+1]= T_in

opti.set_initial(x,x0)
opti.solver('ipopt')
sol = opti.solve()
print(sol.value(x))

RuntimeError: Error in Opti::set_initial [OptiNode] at .../casadi/core/optistack.cpp:127:
.../casadi/core/optistack_internal.cpp:1202: Assertion "v==e[i]" failed:
In initial/value assignment: inconsistent numerical values. At nonzero 48, lhs has 18, while rhs has 0.

Avec scipy : (possible ?)

In [ ]:
def grad(f):
    d = autograd.grad
    def grad_f(x, y):
        return np.array([d(f, 0)(x, y), d(f, 1)(x, y)])
    return grad_f

def Jac(f):
    j = autograd.jacobian
    def J_f(x, y):
        return np.array([j(f, 0)(x, y), j(f, 1)(x, y)]).T
    return J_f

In [ ]:
def f_min(x):
    np.dot(c,x)
    
grad_f_min=grad(f_min)
J_cin= Jac(c_in)
J_ceq= Jac(c_eq)

x0 = np.zeros(2*N+2)
x[N+1]= T_in
    
ineq_cons = {'type': 'ineq','fun' : lambda x: c_in(x),'jac' : lambda x: J_cin(x)}
eq_cons={'type': 'eq','fun' : lambda x: c_eq(x),'jac' : lambda x: J_ceq(x)}
#res = optimize.minimize(f, x0, method='SLSQP', jac=grad_f_min,
#constraints=[ineq_cons, eq_cons], options={'disp': True})

#print(res.x)

### Partie 3

In [11]:
#Si on veut minimiser le coût total des nl logements:
def f_bis(x):
    x_T=np.transpose(x)
    couts= np.matmul(x_T,c)
    return np.sum(couts)

In [12]:
Tm_1=18
Tm_2= 20
TM=30
PM=5000
c_cr=1
c_pl=3/2

In [15]:
a=np.full(2*(2*N+2), c_pl)

a[2:15]=c_cr
a[26:31]=c_cr
a[48:] = 0
c=np.concatenate((a,a))
#print(c)

In [3]:
#j=1 ou 2
#x est la concaténation des vecteurs associés à chaque logement; x est de taille (2N+2)*2= 4N+4, les 2N+2 premières coordonnées
#correspondent au logement 1 (j=1) -pressions et températures - et les 2N+2 dernières au logement 2 (j=2)

def c_eq_bis(x):
    c=np.zeros((2*N+2,2))
    t=t0
    for j in (1,3):
        if j==1:
            T_in=Tm_1
            indice_autre=2
        else:
            T_in=Tm_2
            indice_autre=1
        
        c[N][j-1]=x[N + (2*N+2)*(j-1)]
        c[N+1][j-1]= x[N+1+(2*N+2)*j] - T_in
        
        for i in range(N+2,2*N+2):
            Te= 4+ 8*np.exp(-(t-12)**2/40)
            g=np.exp(-(k+ 2*h)*dt)*x[i-1 +(2*N+2)*(j-1)] +(1-np.exp(-(k+2*h)*dt))(b*x[i-1-(N+1) +(2*N+2)*(j-1)]+h*Te+h*x[i + (2*N+2)*(indice_autre -1)])/(k+2*h)
            c[i][j-1]=x[i +(2*N+2)*(j-1)]- g
            t+=dt
    return c

def c_in_bis(x):
    u1=np.zeros(2*N+2)
    v1=np.zeros(2*N+2)
    u2=np.zeros(2*N+2)
    v2=np.zeros(2*N+2)
    cin=np.zeros((2*N+2,4))
    for i in range(N+1, 2*N+2):
        if (16+N+1<=i and i<=20+N+1):
            v1[i]=Tm - x[i]
            u1[i]=x[i]- TM
            v2[i]=Tm - x[i+(2*N+2)]
            u2[i]=x[i+(2*N+2)]- TM
        else:
            if (38+N+1<=i and i<=47+N+1):
                v1[i]=Tm_1 - x[i]
                u1[i]=x[i]- TM
                v2[i]=Tm_2 - x[i+(2*N+2)]
                u2[i]=x[i+(2*N+2)]- TM
    for i in range(0,N+1):
        u1[i]=x[i]- PM
        v1[i]= - x[i]
        u2[i]=x[i+(2*N+2)]- PM
        v2[i]= - x[i+(2*N+2)]
    cin[:,0]= u1
    cin[:,0]= v1
    cin[:,0]= u2
    cin[:,0]= v2
    return cin

#### Question 9

In [ ]:
epsilon=0.01

In [ ]:
def f_tres(x):
    x_T=np.transpose(x)
    cout= np.matmul(x_T,c)
    cout_final=cout
    for j in range (1,3):
        for i in range (N+1):
            cout_final+=dt*c[i]*c[i]*x[i +(N+1)*(j-1)] + epsilon*(x[i +(N+1)*(j-1)]**2)
    return cout_final

In [ ]:
#nl=2
def solveur (f,x0):
    opti = casadi.Opti()
    n = 4*N+4
    x = opti.variable(n)

    c1=c_in_bis(x)
    
    opti.minimize(f)

    opti.subject_to(c1[:,0]<=0)
    opti.subject_to(c1[:,1]<=0)
    opti.subject_to(c1[:,2]<=0)
    opti.subject_to(c1[:,3]<=0)
    
    c2=c_eq_bis(x)
    opti.subject_to(c2[:,0]==0)
    opti.subject_to(c2[:,1]==0)

    opti.set_initial(x,x0)
    opti.solver('ipopt')
    sol = opti.solve()
    return sol.value(x)

In [ ]:
p= 4*N+4 #taille de x
n=2
def fj(x,j): #j=1 ou 2
    xj=x[2*(N+1)*(j-1):, 2*(N+1) + 2*(N+1)*(j-1)]
    cj=c[2*(N+1)*(j-1):, 2*(N+1) + 2*(N+1)*(j-1)]
    xj_T=np.transpose(xj)
    cout= np.matmul(xj_T,cj)
    cout_final_j=cout
        for i in range (N+1):
            cout_final_j+=dt*c[i]*c[i]*x[i +(N+1)*(j-1)] + epsilon*(x[i +(N+1)*(j-1)]**2) 
    return cout_final_j

def cj(x,j):
    c1=c_eq_bis(x)
    c2=c_in_bis(x)
    if j==1:
        cj=np.concatenate(c1[:,0], c2[:,0],c2[:,1])
    else:
        cj=np.concatenate(c1[:,1],c2[:,2], c2[:,3])
    return cj

In [ ]:
#lambda est de taille 3*(2N+2)

def decomposition_coordination(fun, x0, lambda_0, rho=0.1,N_iter=40,epsilon=0.01): 
    m=len(lambda_0)
    k=0
    xk=x0
    lambda_ki=lambda_0
    lambda_kf=np.maximum(np.array([0]*m), lamb + rho*c(x))
    while np.absolute(lambda_kf-lambda_ki) >=epsilon and (k+1) <N_iter:
        k+=1
        lambda_ki=lambda_kf
        a=xk
        for j in range (1,3):
            xj=xk[2*(N+1)*(j-1):, 2*(N+1) + 2*(N+1)*(j-1)]
            f=fj(xj)+np.matmul(np.transpose(lambda_ki),cj(a,j))
            xk[j-1]=solveur(f,a)
        lambda_kf=np.maximum(np.array([0]*m), lambda_ki + rho*(cj(xk,1)+ cj(xk,2))
    return xk,k 

In [ ]:
x0=
lambda_0=
x,k=decomposition_coordination(f_tres, x0, lambda_0, rho=0.1,N_iter=40,epsilon=0.01)